## Example usage of `adversarial_gym` wrappers using OpenAI Gym and `stable-baselines3`

Import required packages.

`gym` is needed for RL environments.
We use PPO from stable-baselines.

`adversarial_gym` contains 3 wrappers at the moment. One each for adding nosie to observations, actions, and reward. Each of these wrappers is imported.

In [1]:
%load_ext autoreload
%autoreload 2
import gym
from adversarial_gym.wrappers import ObsNoiseWrapper, ActionNoiseWrapper, RewardNoiseWrapper
from stable_baselines3 import PPO

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display

Create the environment.

In [3]:
env = gym.make("CartPole-v1")

Let's try training a PPO agent on this version of the environment without noise so we can see what impact, if any, the noise we add to the next version of the environment will have on performance.

In [4]:
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.6     |
|    ep_rew_mean     | 21.6     |
| time/              |          |
|    fps             | 2916     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 25.3        |
|    ep_rew_mean          | 25.3        |
| time/                   |             |
|    fps                  | 1762        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009394404 |
|    clip_fraction        | 0.107       |
|    clip_range           | 0.2         |
|    entropy_loss   

Wrap the environment in an observation noise wrapper. This wrapper will sample noise from a normal distribution with a mean of zero and a standard deviation of 1.

These parameters can be changed to make the noise more challenging, for example you can change the mean (which will immediately make the environment much more challenging compared to a 0 mean) or increase the standard deviation so there is greater spread to the noise.

You can also try a different noise type. The available options are:
- `"normal"`
- `"uniform"`
- `"discrete_uniform"`
- `"beta"`
- `"poisson"`

Each distribution has the same keyword arguments as the NumPy functions of the same signature (e.g `"normal"` has the same arguments as `numpy.random.normal` and so on).

In [5]:
env = ObsNoiseWrapper(env, "normal", dict(loc=0, scale=1))

Now, we can use stable-baselines to train on the environment for 10000 timesteps.

In [6]:
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22       |
|    ep_rew_mean     | 22       |
| time/              |          |
|    fps             | 2459     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 22           |
|    ep_rew_mean          | 22           |
| time/                   |              |
|    fps                  | 1633         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0075044963 |
|    clip_fraction        | 0.0557       |
|    clip_range           | 0.2          |
|    en

Even though we only ran for 10,000 timesteps, which is not enough to train a model to convergence, we can already see the impact that the added noise is having on the training process. The model trained on the environment *without* noise has already obtained a reward of 65.5 at timestep 10240, while the model trained on the *noisy* environment only has a reward of 33.8.

The `ActionNoiseWrapper` and `RewardNoiseWrapper` work in the same way as the `ObsNoiseWrapper`demoed here.